In [ ]:
//import sys.process._
//"wget https://github.com/adrianulbona/osm-parquetizer/releases/download/v1.0.0/osm-parquetizer-1.0.0.jar -P /FileStore/group14" !!

//import sys.process._
//"wget http://download.geofabrik.de/europe/sweden-latest.osm.pbf -P dbfs:/FileStore/group14" !!

//import sys.process._
//"java -jar /FileStore/group14/osm-parquetizer-1.0.0.jar /FileStore/group14/sweden-latest.osm.pbf" !!



In [ ]:
import org.apache.spark.sql._
import scala.sys.process._
import org.apache.spark.sql.functions.{col}

def toMap(tupesArray: Seq[Row]): Option[Map[String, String]] = {
    if (tupesArray == null) {
      None
    } else {
      val tuples = tupesArray.map(e => {
        (
          e.getAs[String]("key"),
          e.getAs[String]("value")
        )
      })
      Some(tuples.toMap)
    }
  }

def handleCommon()(df:DataFrame):DataFrame = {
  val toMapUDF = udf(toMap _)
  df.drop("uid", "user_sid", "changeset", "version", "timestamp")
    .withColumn("tags", toMapUDF(col("tags")))
}


  

>     import org.apache.spark.sql._
>     import scala.sys.process._
>     import org.apache.spark.sql.functions.col
>     toMap: (tupesArray: Seq[org.apache.spark.sql.Row])Option[Map[String,String]]
>     handleCommon: ()(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame

In [ ]:
sqlContext.setConf("spark.sql.parquet.binaryAsString","true")
val nodeDF = sqlContext.read.parquet("dbfs:/FileStore/group14/sweden-latest_osm_pbf_node.parquet").transform(handleCommon())
nodeDF.createOrReplaceTempView("nodes")

val wayDF = sqlContext.read.parquet("dbfs:/FileStore/group14/sweden_latest_osm_pbf_way.parquet").transform(handleCommon())
wayDF.createOrReplaceTempView("ways")

//val relationDF = sqlContext.read.parquet("dbfs:/FileStore/group14/sweden_latest_osm_pbf_relation.parquet").transform(handleCommon())
//relationDF.createOrReplaceTempView("relations")
print(wayDF.columns.toSeq)

  

>     WrappedArray(id, tags, nodes)nodeDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, tags: map<string,string> ... 2 more fields]
>     wayDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, tags: map<string,string> ... 1 more field]

In [ ]:
In order to reduce the number of nodes and 

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions.{udf, explode,arrays_zip,lag, lead, concat,array, lit}

//val wayDF_tmp2 = wayDF_tmp.withColumn("exploded", explode(arrays_zip($"nodes", $"nodes".drop(1))))
//wayDF_tmp1.withColumn("tesrasrdasd", concat($"nodes",$"nullcol"))
val wayDF_exploded = wayDF.withColumn("exploded", explode(arrays_zip(concat($"nodes.nodeId",array(lit(-1L))), concat(array(lit(-1L)),$"nodes.nodeId"))))
val wayDF_filtered = wayDF_exploded.filter($"exploded.0" > 0 && $"exploded.1" > 0)

  

>     import org.apache.spark.sql._
>     import org.apache.spark.sql.functions.{udf, explode, arrays_zip, lag, lead, concat, array, lit}
>     wayDF_exploded: org.apache.spark.sql.DataFrame = [id: bigint, version: int ... 4 more fields]
>     wayDF_filtered: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, version: int ... 4 more fields]

In [ ]:
val wayDF_filtered = wayDF_exploded.filter($"exploded.0" > 0 && $"exploded.1" > 0)

  

>     wayDF_filtered: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: bigint, version: int ... 5 more fields]

In [ ]:
wayDF_filtered.select("exploded").take(50)

  

>     res27: Array[org.apache.spark.sql.Row] = Array([[2416576872,2416576873]], [[5159477197,2416576872]], [[2416576867,5159477197]], [[5159477198,2416576867]], [[2416576866,5159477198]], [[5159477199,2416576866]], [[1960564960,5159477199]], [[5159477196,1960564960]], [[5915403101,5159477196]], [[97772279,5915403101]], [[1960564965,2416576876]], [[5159477214,1960564965]], [[5159477215,5159477214]], [[1960564961,5159477215]], [[5159477217,1960564961]], [[5159477218,5159477217]], [[1960564958,5159477218]], [[2416576873,1960564958]], [[5159477329,2416576873]], [[2416576864,5159477329]], [[2416579043,2416579038]], [[2416579045,2416579043]], [[2416579046,2416579045]], [[2416579038,2416579046]], [[2416579127,2416579021]], [[2416579024,2416579127]], [[2416579163,2416579024]], [[2416579021,2416579163]], [[2416579054,2416579146]], [[2416579067,2416579054]], [[2416586043,2416579067]], [[2416579184,2416586043]], [[2416579146,2416579184]], [[2416579099,2416579104]], [[2416594473,2416579099]], [[2416579069,2416594473]], [[2416579136,2416579069]], [[2416579104,2416579136]], [[2416579161,2416579029]], [[2416579097,2416579161]], [[2416579072,2416579097]], [[2416579029,2416579072]], [[2416579200,2416579036]], [[2416579198,2416579200]], [[2416582165,2416579198]], [[2416579025,2416582165]], [[2416579036,2416579025]], [[2416579065,2416579059]], [[2416609471,2416579065]], [[2416579144,2416609471]])

In [ ]:
wayDF_exploded.select("exploded").take(50)

  

>     res26: Array[org.apache.spark.sql.Row] = Array([[2416576873,-1]], [[2416576872,2416576873]], [[5159477197,2416576872]], [[2416576867,5159477197]], [[5159477198,2416576867]])

In [ ]:
 paths

In [ ]:
import org.apache.spark.sql.functions._

val wayNodeDF = wayDF_exploded.select($"exploded.0".as("start"), $"exploded.1".as("end"),$"tags.highway", $"tags.maxspeed")
.filter($"highway" isin ("motorway","trunk","primary","secondary", "tertiary", "unclassified", "residential","motorway_link", "trunk_link",  "primary_link", "secondary_link", "tertiary_link"))
//  .filter(array_contains($"tags.value", "motorway") || array_contains($"tags.value", "trunk") || array_contains($"tags.value", "primary") || array_contains($"tags.value", " 	secondary") || array_contains($"tags.value", "tertiary") || array_contains($"tags.value", "unclassified")  || array_contains($"tags.value", "residential") || array_contains($"tags.value", "motorway_link") || array_contains($"tags.value", "trunk_link") || array_contains($"tags.value", "primary_link") || array_contains($"tags.value", "secondary_link") || array_contains($"tags.value", "tertiary_link"))
  //.select($"id".as("wayId"), $"user_sid", explode($"nodes").as("indexedNode"))
wayNodeDF.createOrReplaceTempView("wayHighway")
//val rdd = wayNodeDF.rdd

  

>     import org.apache.spark.sql.functions._
>     wayNodeDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [start: bigint, end: bigint ... 2 more fields]

  

We then add default speeds to each edge. If it is a motorway it gets a
speed of 110km/h, residential areas gets 15 km/h while all other gets 50
km/h.

In [ ]:
val wayNodeDF_nonull = wayNodeDF.withColumn("maxspeed", when($"maxspeed".isNull && col("highway") == "motorway", 110)
                     .when($"maxspeed".isNull && col("highway")=="primary", 50).when($"maxspeed".isNull && col("highway")=="secondary", 50).when($"maxspeed".isNull && col("highway")=="motorway_link", 50)
                     .when($"maxspeed".isNull && col("highway")=="residential", 15).when($"maxspeed".isNull, 50)
                     .otherwise($"maxspeed"))
wayNodeDF_nonull.createOrReplaceTempView("wayHighway")


  

>     wayNodeDF_nonull: org.apache.spark.sql.DataFrame = [start: bigint, end: bigint ... 2 more fields]

In [ ]:
import org.apache.spark.sql.functions._

val nodeLatLonDF = nodeDF
  .select($"id".as("nodeId"), $"latitude".as("startLat"), $"longitude".as("startLong"))

val endnodeLatLonDF = nodeDF
  .select($"id".as("nodeId2"), $"latitude".as("endLat"), $"longitude".as("endLong"))

val wayGeometryDF = wayNodeDF_nonull.join(nodeLatLonDF, $"start" === $"nodeId").join(endnodeLatLonDF, $"end" === $"nodeId2")

val wayGeometry_distDF = wayGeometryDF.withColumn("a", pow(sin(radians($"endLat" - $"startLat") / 2), 2) + cos(radians($"startLat")) * cos(radians($"endLat")) * pow(sin(radians($"endLong" - $"startLong") / 2), 2))
  .withColumn("distance", atan2(sqrt($"a"), sqrt(-$"a" + 1)) * 2 * 6371)
  .filter($"endLat"<55.4326186d) //South of malmö
  .withColumn("time", $"distance"/$"maxspeed").select("time", "start", "end", "distance", "maxspeed")
wayGeometry_distDF.createOrReplaceTempView("wayGeometry_distDF")

  

>     import org.apache.spark.sql.functions._
>     nodeLatLonDF: org.apache.spark.sql.DataFrame = [nodeId: bigint, startLat: double ... 1 more field]
>     endnodeLatLonDF: org.apache.spark.sql.DataFrame = [nodeId2: bigint, endLat: double ... 1 more field]
>     wayGeometryDF: org.apache.spark.sql.DataFrame = [start: bigint, end: bigint ... 8 more fields]
>     wayGeometry_distDF: org.apache.spark.sql.DataFrame = [time: double, start: bigint ... 3 more fields]

In [ ]:
//import org.graphframes._
//val g = GraphFrame(
  //verexDf.select($"id", $"name"), 
 // edgeDf.select ($"sourceID" as "src", $"destID" as "dst", $"relationship"))
// Convert to GraphX
//val gx: Graph[Row, Row] = g.toGraphX

In [ ]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
val eps = 0.000001
val edges: RDD[Edge[Double]]  =  wayGeometry_distDF
    .select("start", "end", "time").rdd.map(line => Edge(line.getAs("start"), line.getAs("end"), line.getAs("time")))
val revedges: RDD[Edge[Double]]  =  wayGeometry_distDF
    .select("start", "end", "time").rdd.map(line => Edge(line.getAs("end"), line.getAs("start"), line.getAs("time")))

val graphorg = Graph.fromEdges(edges, "defaultname")
graphorg.cache()

val revgraph = Graph.fromEdges(revedges, "defaultname")
//val edge = edges.toDF("id", "node")

val graph = Graph(
   graphorg.vertices,
    graphorg.edges.union(revgraph.edges)
).groupEdges((attr1, attr2) => scala.math.max(eps, scala.math.min(attr1, attr2)))

graph.cache()
println(graphorg.edges.toDF.count())
println(graphorg.vertices.toDF.count())

  

>     25402
>     24148
>     import org.apache.spark.graphx._
>     import org.apache.spark.rdd.RDD
>     eps: Double = 1.0E-6
>     edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Double]] = MapPartitionsRDD[472993] at map at command-1716517738645653:5
>     revedges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Double]] = MapPartitionsRDD[473113] at map at command-1716517738645653:7
>     graphorg: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@48b6952b
>     revgraph: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@69df44ba
>     graph: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@2a139bec

In [ ]:
import scala.reflect.ClassTag
import org.apache.spark.graphx._

/*  
 * Computes shortest weighted paths to the given set of landmark vertices, returning a graph where each
 * vertex attribute is a map containing the shortest-path distance to each reachable landmark.
 * Currently supports only Graph of [VD, Double], where VD is an arbitrary vertex type.
 *
 * The object also include a function which transforms the resulting graph into a path_graph between a 
 * specific starting node and goal node. Each edge in the path_grpah is either 1 or 0 depending if it is 
 * the shortest path or not.
 *
 */
object ShortestPath extends Serializable {

  // When finding the shortest path each node stores a map from the itself to each goal node.
  // The map returns an array includeing the total distance to the goal node as well as the
  // next node pn the shortest path to the goal node. The last value in the array is only 
  // populated with the nodes own id and is only used for computational convenience. 
  type SPMap = Map[VertexId, Tuple3[Double, VertexId, VertexId]]
  
  // PN holds the information of the path nodes which are used for creating a path graph
  // PN = ('Distance left to goal node', 'Next path node id', 'Goal node', 'Is on path')
  type PN = Tuple4[Double, VertexId, VertexId, Boolean] 
  
  private val INITIAL_DIST = 0.0
  private val DEFAULT_ID = -1L
  private val INFINITY = Int.MaxValue.toDouble

  private def makeMap(x: (VertexId, Tuple3[Double, VertexId, VertexId])*) = Map(x: _*)
  
  private def incrementMap(spmap: SPMap, delta: Double, id: VertexId): SPMap = { 
    spmap.map { case (v, d) => v -> (Tuple3(d._1 + delta, d._3, id)) }
  }

  private def addMaps(spmap1: SPMap, spmap2: SPMap): SPMap = {
    (spmap1.keySet ++ spmap2.keySet).map {
    k =>{
        if (spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1 < spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1) 
                k -> (Tuple3(spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1, 
                             spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._2, 
                             spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._3))
        else 
                k -> (Tuple3(spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1, 
                             spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._2, 
                             spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._3))
        }
    }.toMap
  }
  
  // at this point it does not really matter what vertex type is
  def run[VD](graph: Graph[VD, Double], landmarks: Seq[VertexId]): Graph[SPMap, Double] = {
    val spGraph = graph.mapVertices { (vid, attr) =>
      // initial value for itself is 0.0 as Double
      if (landmarks.contains(vid)) makeMap(vid -> Tuple3(INITIAL_DIST, DEFAULT_ID, DEFAULT_ID)) else makeMap()
    }

    val initMaps = makeMap()

    def vProg(id: VertexId, attr: SPMap, msg: SPMap): SPMap = {
      addMaps(attr, msg)
    }

    def sendMsg(edge: EdgeTriplet[SPMap, Double]): Iterator[(VertexId, SPMap)] = {
      val newAttr = incrementMap(edge.dstAttr, edge.attr, edge.srcId)
      if (edge.srcAttr != addMaps(newAttr, edge.srcAttr)) Iterator((edge.srcId, newAttr))
      else Iterator.empty
    }

    Pregel(spGraph, initMaps)(vProg, sendMsg, addMaps)
  }
  
  def create_path_graph[VD](graph: Graph[SPMap, Double], goalId: VertexId, startId: VertexId): Graph[PN, Int] = {
    // For a given goal node we remove the lookup map and extend the state to a Tuple5 with the goal id and a boolean
    val path = graph.mapEdges(e => 0)
              .mapVertices((vertixId, attr) => {
                if (attr.contains(goalId)) {
                  val path_step = attr(goalId)
                  if (vertixId == path_step._3 && path_step._2 == -1L)
                    (path_step._1, goalId, goalId, false) // while we are at it, we clean up the state a bit
                  else  
                    (path_step._1, path_step._2, goalId, false)
                } else// If the vertice does not have a map to our goal we add a default value to it
                    (INFINITY, -1L, -1L, false)
              })

      def mergeMsg(msg1: VertexId, msg2: VertexId): VertexId = { // we should only get one msg
          msg2
      }

      def vprog(id: VertexId, attr: PN, msg: VertexId): PN = {
        // Check that the current node is the one adressed in the message
        if (id == msg)
          (attr._1, attr._2, attr._3, true)
        else // If the message is not addressed to the current node (happens for inital message), use the old value 
          attr
      }
      def sendMsg(triplet: EdgeTriplet[PN, Int]): Iterator[(VertexId, VertexId)] = {
        // If dstId is the next node on the path and has not yet been activated
        if (triplet.srcAttr._2 == triplet.dstId && triplet.srcAttr._4 && !triplet.dstAttr._4) 
          Iterator((triplet.dstId, triplet.dstId))// Send next msg
        else
          Iterator.empty// Do nothing
      }

      Pregel(path, startId)(vprog, sendMsg, mergeMsg).mapTriplets(triplet => {
        if(triplet.srcAttr._2 == triplet.dstId && triplet.srcAttr._4)
          1
        else
          0
      })
  }
}

println("Usage: val result = GraphXShortestWeightedPaths.run(graph, Seq(4L, 0L, 9L))")

  

>     Usage: val result = GraphXShortestWeightedPaths.run(graph, Seq(4L, 0L, 9L))
>     import scala.reflect.ClassTag
>     import org.apache.spark.graphx._
>     defined object ShortestPath

In [ ]:
val cc = graph.connectedComponents()
val ccDf = cc.vertices.toDF
ccDf.groupBy($"_2").count().collect()

In [ ]:
val start = 2416576872L
val goal = 2416609471L
val landMarkVertexIds = Seq(goal)
val result = ShortestPath.run(graph, landMarkVertexIds)

//println(path_graph.vertices.collect.mkString("\n"))
//println(path_graph.edges.collect.mkString("\n"))


  

>     start: Long = 2416576872
>     goal: Long = 2416609471
>     landMarkVertexIds: Seq[Long] = List(2416609471)
>     result: org.apache.spark.graphx.Graph[ShortestPath.SPMap,Double] = org.apache.spark.graphx.impl.GraphImpl@72ea9eec

In [ ]:
val path_graph = ShortestPath.create_path_graph(result, goal, start)


  

>     path_graph: org.apache.spark.graphx.Graph[ShortestPath.PN,Int] = org.apache.spark.graphx.impl.GraphImpl@7128ebb

In [ ]:
display(result.vertices.toDF.filter)

In [ ]:
display(graph.edges.toDF)

In [ ]:
//val path_df = path_graph.edges.toDF
//display(path_df)
//val path_df2 = path_graph.edges.toDF.filter(col("attr")>0)
//display(path_df2)



  

>     1016395

In [ ]:
print(graph.vertices.toDF.count())

  

>     736490

In [ ]:
select count(*) from wayGeometry_distDF

In [ ]:
import com.vividsolutions.jts.io.WKTWriter
import com.vividsolutions.jts.geom.{Coordinate, Geometry, GeometryFactory}

case class IndexedNode(index: Int, coord: Coordinate)

def nodesToWKT(nodes: Seq[Row]): String = {
    val indexedCoords = nodes.map { 
      case Row(index: Int, lat: Double, lon:Double) => IndexedNode(index, new Coordinate(lon, lat))
    }
    val coords = indexedCoords.sortBy(_.index).map(_.coord).toArray
    new WKTWriter().write(new GeometryFactory().createLineString(coords))
}

val wkt = udf {
  (nodes: Seq[Row]) => nodesToWKT(nodes)
}

val wayWKTDF = wayGeometryDF
  .filter(size($"geometry") > 1)
  .select($"wayId", $"user_sid", wkt($"geometry").as("geometry"))
wayWKTDF.createOrReplaceTempView("wayWKT")

  

>     import com.vividsolutions.jts.io.WKTWriter
>     import com.vividsolutions.jts.geom.{Coordinate, Geometry, GeometryFactory}
>     defined class IndexedNode
>     nodesToWKT: (nodes: Seq[org.apache.spark.sql.Row])String
>     wkt: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StringType,None)
>     wayWKTDF: org.apache.spark.sql.DataFrame = [wayId: bigint, user_sid: string ... 1 more field]